# Unstructured API Walkthrough

This walkthrough provides you with deep, hands-on experience with the [Unstructured API](https://docs.unstructured.io/api-reference/overview). As you follow along, you will learn how to use many of Unstructured's features for
[partitioning](https://docs.unstructured.io/ui/partitioning),
[enriching](https://docs.unstructured.io/ui/enriching/overview),
[chunking](https://docs.unstructured.io/ui/chunking),
and [embedding](https://docs.unstructured.io/ui/embedding).
These features are optimized for turning your source documents and data into information that is well-tuned for
[retrieval-augmented generation (RAG)](https://unstructured.io/blog/rag-whitepaper),
[agentic AI](https://unstructured.io/problems-we-solve#powering-agentic-ai),
and [model fine-tuning](https://www.geeksforgeeks.org/deep-learning/what-is-fine-tuning/).

This walkthrough uses two sample files to demonstrate how Unstructured identifies and processes content such as image, graphs, complex tables, non-English characters, and handwriting. These files, which are available for you to download to your local machine, include:

- Wang, Z., Liu, X., & Zhang, M. (2022, November 23). _Breaking the Representation Bottleneck of Chinese Characters: Neural Machine Translation with Stroke Sequence Modeling_. arXiv.org. https://arxiv.org/pdf/2211.12781. This 12-page PDF file features English and non-English characters, images, graphs, and complex tables. Throughout this walkthrough, this file's title is shortened to “Chinese characters” for brevity.
- United States Central Security Service. (2012, January 27). _National Cryptologic Museum Opens New Exhibit on Dr. John Nash_. United States National Security Agency. https://courses.csail.mit.edu/6.857/2012/files/H03-Cryptosystem-proposed-by-Nash.pdf. This PDF file features English handwriting and scanned images of documents. Throughout this walkthrough, this file's title is shortened to “Nash letters” for brevity.

If you are not able to complete any of the following steps, contact Unstructured Support at [support@unstructured.io](mailto:support@unstructured.io).

---
>
> 💡 _What's this?_
>
> As you move through this walkthrough, you will notice tips like this one.
> These tips are designed to help expand your knowledge about Unstructured as
> you go. Feel free to skip these tips for now if you are in a hurry. You can
> always return to them later to learn more.
>
---

## Requirements

To run this notebook, you will need:

- An Unstructured account. To sign up for an account, go to https://unstructured.io/?modal=try-for-free. After you sign up, you are immediately signed in to your new account, at https://platform.unstructured.io.
- An Unstructured API key, as follows:

  1. After you are signed in to your account, on the sidebar click **API Keys**.
  2. Click **Generate New Key**.
  3. Enter some meaningful display name for the key, and then click **Continue**.
  4. Next to the new key's name, click the **Copy** icon. The key's value is copied to your system's clipboard. If you lose this key, simply return to the list and click **Copy** again.

- One or more local files for Unstructured to process. This notebook assumes that the local files you want to process are in a subfolder that is accessible from this notebook. The easiest and fastest way to create this subfolder is as follows:

  1. On this notebook's sidebar, click the folder (**Files**) icon. By default, the **Files** pane will show all files and subfolders within the `/content` folder.
  2. Right-click in a blank area anywhere below the existing list of subfolders and files in the **Files** pane, and then click **New folder**.
  3. Enter a name for the new subfolder within `/content`. This notebook assumes the subfolder is named `input`.
  4. To upload files to this subfolder, do the following:

     a. Rest your mouse pointer on the `input` subfolder.<br/>
     b. Click the ellipsis (three dots) icon, and then click **Upload**.<br/>
     c. Browse to and select the "Chinese characters" and "Nash letters" files on your local machine that you want to upload to this `input` subfolder.<br/>

- A destination subfolder for Unstructured to send its processed results to. This notebook assumes that the destination folder is accessible from this notebook. The easiest and fastest way to create this folder is as follows:

  1. If the **Files** pane is not already open, then on this notebook's sidebar, click the folder (**Files**) icon. By default, the **Files** pane will show all files and subfolders within the `/content` folder.
  2. Right-click in a blank area anywhere below the existing list of subfolders and files in the **Files** pane, and then click **New folder**.
  3. Enter a name for the new subfolder within `/content`. This notebook assumes the subfolder is named `output`.

---
>
> ⚠️ **Warning**: Any files that you upload to these `input` or `output`
> subfolders will be deleted whenever Google Colab disconnects or resets, for
> example due to inactivity, manual restart, or session timeout.
>
---

## Step 1: Create a custom workflow

In this step, you create a custom [workflow](https://docs.unstructured.io/api-reference/workflow/workflows) in your Unstructured account.
Workflows are defined sequences of processes that automate the flow of data from your source documents and data into Unstructured for processing. Unstructured then sends its processed data over into your destination file storage locations, databases, and vector stores. Your RAG apps, agents, and models can then use this processed data in those destinations to do things more quickly and accurately such as
[answering users' questions](https://learn.microsoft.com/en-us/azure/developer/ai/advanced-retrieval-augmented-generation),
[automating business processes](https://unstructured.io/problems-we-solve#business-process-automation),
and [expanding your organization's available body of knowledge](http://knowledgemanagement.ie/the-critical-role-of-knowledge-management-as-a-foundation-for-llms-and-ai/).

---
>
> 💡 _Which kinds of sources and destinations does Unstructured support?_
>
> Unstructured can connect to many types of sources and destinations including
> file storage services such as Amazon S3 and Google Cloud Storage; databases
> such as PostgreSQL; and vector storage and database services such as MongoDB
> Atlas and Pinecone.
>
> See the full list of [supported source connectors](https://docs.unstructured.io/api-reference/workflow/sources/overview) and [supported destination connectors](https://docs.unstructured.io/api-reference/workflow/destinations/overview).
>
---
>
> 💡 _Which kinds of files does Unstructured support?_
>
> Unstructured can process a wide variety of file types including PDFs, word
> processing documents, spreadsheets, slide decks, HTML, image files, emails,
> and more.
>
> See the full list of [supported file types](https://docs.unstructured.io/api-reference/supported-file-types).
>
---

1. In the following cell, replace `<your-unstructured-api-key>` with your pasted Unstructured API key's value and then run the cell, which sets the constant `UNSTRUCTURED_API_KEY` to your API key's value.

In [ ]:
UNSTRUCTURED_API_KEY = "<your-unstructured-api-key>"

2. Run the following cell, which installs the `unstructured-client` and `datetime` libraries. These libraries are required for making HTTP requests to the Unstructured API and for assigning unique asset names in this example, respectively.

In [ ]:
!pip install -U unstructured-client datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.7/264.7 kB 17.2 MB/s eta 0:00:00


3. Run the following cell, which creates an Unstructured custom workflow with a single node for partitioning. This workflow uses the **High Res** [partitioning](https://docs.unstructured.io/ui/partitioning) strategy to turn the contents of your documents and semi-structured data into a data format that is fine-tuned for well-tuned for retrieval-augmented generation (RAG), agentic AI, and model fine-tuning.

   [Learn more about this code](https://docs.unstructured.io/api-reference/workflow/overview#create-a-workflow).

In [ ]:
from datetime import datetime

from unstructured_client import UnstructuredClient
from unstructured_client.models.shared import (
    WorkflowNode,
    CreateWorkflow,
    WorkflowType
)
from unstructured_client.models.operations import CreateWorkflowRequest

with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    partition_node = WorkflowNode(
        name="Partitioner",
        subtype="unstructured_api",
        type="partition",
        settings={
            "strategy": "hi_res"
        }
    )

    workflow = CreateWorkflow(
        name=f"api-demo-workflow-{datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}",
        workflow_type=WorkflowType.CUSTOM,
        workflow_nodes=[
            partition_node
        ]
    )

    response = client.workflows.create_workflow(
        request=CreateWorkflowRequest(
            create_workflow=workflow
        )
    )

    print(f"Workflow details:\n---\n{response.workflow_information.model_dump_json(indent=4)}")

    WORKFLOW_ID = response.workflow_information.id

Workflow details:
---
{
    "created_at": "2025-09-30T18:04:14.558164Z",
    "destinations": [],
    "id": "ca99f800-8746-43e1-8f92-b6e36dbd22f8",
    "name": "api-demo-workflow-2025-09-30-18-04-14",
    "sources": [],
    "status": "active",
    "workflow_nodes": [
        {
            "name": "Partitioner",
            "subtype": "unstructured_api",
            "type": "partition",
            "id": "1931bdfa-0901-4d58-baaf-c710fa4b34b3",
            "settings": {
                "strategy": "hi_res",
                "include_page_breaks": false,
                "pdf_infer_table_structure": false,
                "exclude_elements": null,
                "xml_keep_tags": false,
                "encoding": "utf-8",
                "extract_image_block_types": [
                    "image",
                    "table"
                ]
            }
        }
    ],
    "reprocess_all": true,
    "schedule": {
        "crontab_entries": []
    },
    "updated_at": "2025-09-30T18:04:14

---
>
> 💡 _What other kinds of settings can I set for my workflow?_
>
> Your workflow can be set to run automatically (on a regular time schedule).
> You can also set your workflow to use a paritioning strategy such as
> **Auto**, **Fast**, or **VLM**, instead of **High Res**.
> [Learn how](https://docs.unstructured.io/api-reference/workflow/workflows).
>
---
>
> 💡 _When would I choose **Auto**, **Fast**, **High Res**, or **VLM**?_
>
> You'll learn more about these [partitioning](https://docs.unstructured.io/ui/partitioning) strategies starting with **Step 2**. But for now:
>
> - **Auto** is recommended in most cases. It lets Unstructured figure out the
> best strategy to switch over to for each incoming file (and even for each
> page if the incoming file is a PDF), so you don't have to!
> - **Fast** is only for when you know for certain that none of your files have
> tables, images, or multilanguage, scanned, or handwritten content in them.
> It's optimized for partitioning text-only content and is the fastest of all
> the strategies. It can recognize the text for only a few languages other than
> English.
> - **High Res** is only for when you know for certain that at least one of your
> files has images or simple tables in them, and that none of your files also
> have scanned or handwritten content in them. It can recognize the text for
> more languages than **Fast** but not as many as **VLM**.
> - **VLM** is great for any file, but it is best when you know for certain that
> some of your files have a combination of tables (especially complex ones),
> images, and multilanguage, scanned, or handwritten content. It's the highest
> quality but slowest of all the strategies.
>
> In this walkthrough, you switch between **High Res** and **VLM** strategies
> only to see how each of these strategies works with a combination of complex
> tables, images, and multilanguage, scanned, and handwritten content. In
> practice, for these kinds of files you would likely just want to choose
> **Auto**.
>
---
>
> 💡 _This workflow has only a partitioner. What about enriching, chunking, and embedding?_
>
> Don't worrry; you will add nodes to this workflow for enriching, chunking,
> and embedding later in this notebook.
>
---

## Step 2: Experiment with partitioning

In this step, you use your new workflow to [partition](https://docs.unstructured.io/ui/partitioning) the sample PDF files that you downloaded earlier onto your local machine. Partitioning is the process where Unstructured identifies and extracts content from your source documents and then outputs this content as a series of contextually-rich document elements and metadata, which are well-tuned for RAG, agentic AI, and model fine-tuning. This step shows how well Unstructured's **High Res** partitioning strategy identifies and extracts content, and how well Unstructured's **VLM** partitioning strategy handles more complex content such as complex tables, multilanguage characters, and handwriting.


1. Run the following cell, which sets the notebook's local input paths in the `/content/input` subfolder for the "Chinese characters" and "Nash letters" files that you uploaded from your local machine. This cell also sets the notebook's local output paths in the `/content/output` subfolder for Unstructured to send its processed data for you to view later.

In [ ]:
INPUT_PATH = "/content/input"
OUTPUT_PATH = "/content/output"
CHINESE_CHARACTERS_FILE_NAME = "2211.12781v1.pdf"
NASH_LETTERS_FILE_NAME = "H03-Cryptosystem-proposed-by-Nash.pdf"

CHINESE_CHARACTERS_INPUT_PATH = f"{INPUT_PATH}/{CHINESE_CHARACTERS_FILE_NAME}"
CHINESE_CHARACTERS_OUTPUT_PATH = f"{OUTPUT_PATH}/{CHINESE_CHARACTERS_FILE_NAME}.json"

NASH_LETTERS_INPUT_PATH = f"{INPUT_PATH}/{NASH_LETTERS_FILE_NAME}"
NASH_LETTERS_OUTPUT_PATH = f"{OUTPUT_PATH}/{NASH_LETTERS_FILE_NAME}.json"

2. Run the following cell, which runs a job that is based on the workflow that you created in **Step 1**. This job is identified by its job ID, which is set programmatically so that you can use it in later cells as needed.

   [Learn more about this code](https://docs.unstructured.io/api-reference/workflow/overview#run-a-workflow).

In [ ]:
from unstructured_client import UnstructuredClient
from unstructured_client.models.operations import RunWorkflowRequest
from unstructured_client.models.shared import InputFiles

def run_workflow(file_path):
    input_files = []

    for filename in [
        file_path
    ]:
        with open(filename, "rb") as f:
            input_files.append(
                InputFiles(
                    content=f.read(),
                    file_name=filename
                )
            )

    with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
        response = client.workflows.run_workflow(
            request={
                "workflow_id": WORKFLOW_ID,
                "body_run_workflow": {
                    "input_files": input_files
                }
            }
        )

    print(f"Job details:\n---\n{response.job_information.model_dump_json(indent=4)}")

    return response.job_information.id

JOB_ID = run_workflow(CHINESE_CHARACTERS_INPUT_PATH)

Job details:
---
{
    "created_at": "2025-09-30T18:05:11.129644Z",
    "id": "c0e0299d-36ce-417e-b98b-f60d63d05eb7",
    "status": "SCHEDULED",
    "workflow_id": "ca99f800-8746-43e1-8f92-b6e36dbd22f8",
    "workflow_name": "api-demo-workflow-2025-09-30-18-04-14",
    "job_type": "ephemeral"
}


3. Run the following cell, which uses the job's ID to get the job's status. Do not proceed until the job polling is complete.

   - If the job's final status is completed but with errors, or is failed, continue ahead to step 4 in this procedure to try to find out why.
   - If the job's final status is stopped, try running the previous cell in step 2 again to produce a new job for this workflow.
   - If the job's final status is successfully completed, skip ahead to step 5 in this procedure.

   [Learn more about this code](https://docs.unstructured.io/api-reference/workflow/overview#get-processing-details-for-a-job).


In [ ]:
import time

from unstructured_client.models.operations import GetJobDetailsRequest

def poll_job_status(job_id):
    with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
        while True:
            response = client.jobs.get_job_details(
                request=GetJobDetailsRequest(
                    job_id=job_id
                )
            )

            job = response.job_details

            if job.processing_status == "SCHEDULED":
                print("Job is scheduled, polling again in 10 seconds...")
                time.sleep(10)
            elif job.processing_status == "IN_PROGRESS":
                print("Job is in progress, polling again in 10 seconds...")
                time.sleep(10)
            elif job.processing_status == "COMPLETED_WITH_ERRORS":
                print("Job completed but with errors. Run the next cell to find out why.")
                break
            elif job.processing_status == "STOPPED":
                print("Job was stopped. No data was generated.")
                break
            elif job.processing_status == "FAILED":
                print("Job failed. Run the next cell to find out why.")
                break
            else:
                print(f"Job successfully completed.")
                break

        return job

job = poll_job_status(JOB_ID)
print(f"Job details:\n---\n{job.model_dump_json(indent=4)}")

Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job successfully completed.
Job details:
---
{
    "id": "c0e0299d-36ce-417e-b98b-f60d63d05eb7",
    "node_stats": [
        {
            "failure": 0,
            "in_progress": 0,
            "ready": 0,
            "success": 1,
            "node_name": "Partitioner",
            "node_subtype": "unstructured_api",
            "node_type": "partition"
        }
    ],
    "processing_status": "SUCCESS",
    "message": null
}


4. If the job is completed but with errors, or is failed, run the following cell to try to find out why. Otherwise, skip ahead to step 5 in this procedure.

   [Learn more about this code](https://docs.unstructured.io/api-reference/workflow/overview#get-failed-file-details-for-a-job).

In [ ]:
from unstructured_client.models.operations import GetJobFailedFilesRequest

def get_failed_files_details(job_id):
    with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
        response = client.jobs.get_job_failed_files(
            request=GetJobFailedFilesRequest(
                job_id=JOB_ID
            )
        )

        info = response.job_failed_files

        if info.failed_files.__len__() > 0:
            print(f"{info.failed_files.__len__()} failed file(s):")

            for failed_file in info.failed_files:
                print(f"---")
                print(f"document: {failed_file.document}")
                print(f"error:    {failed_file.error}")
        else:
            print(f"No failed files.")

get_failed_files_details(JOB_ID)

No failed files.


5. If the job is successfully completed, run the following cell to download the data that Unstructured generated for the "Chinese characters" file to this notebook's local `/content/output` subfolder.

   [Learn more about this code](https://docs.unstructured.io/api-reference/workflow/overview#download-a-processed-local-file-from-a-job).

In [ ]:
import json

from unstructured_client.models.operations import (
    GetJobRequest,
    DownloadJobOutputRequest
)

def get_job_download_details(job_id):
    with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
        response = client.jobs.get_job(
            request=GetJobRequest(
                job_id=job_id
            )
        )

    return response.job_information

def download_processed_file(job_id, file_id, node_id, output_path):
    with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
        response = client.jobs.download_job_output(
            request=DownloadJobOutputRequest(
                job_id=job_id,
                file_id=file_id,
                node_id=node_id
            )
        )

        with open(output_path, "w") as f:
            json.dump(response.any, f, indent=4)

job_info = get_job_download_details(JOB_ID)

download_processed_file(
    job_id=JOB_ID,
    file_id=job_info.output_node_files[0].file_id,
    node_id=job_info.output_node_files[0].node_id,
    output_path=CHINESE_CHARACTERS_OUTPUT_PATH
)

6. You can now look at the data that Unstructured generated. Do this as follows:

   a. On this notebook's sidebar, click the folder (**Files**) icon, if the **Files** pane is not already shown.

   b. In the **Files** pane, click to expand the `output` folder.
   
   c. Double-click the file titled `2211.12781v1.pdf.json`.
   
   d. The file's contents appear in a pane on the right side of this notebook.

---
>
> 💡 _What am I looking at in the output here?_
>
> - Unstructured outputs its results in industry-standard [JSON](https://www.json.org/) format, which is
> ideal for RAG, agentic AI, and model fine-tuning.
> - Each object in the JSON is called a [document element](https://docs.unstructured.io/ui/document-elements) and contains a `text`
> representation of the content that Unstructured detected for the particular
> portion of the document that was analyzed.
> - The `type` is the kind of document element that Unstructured categorizes it
> as, such as whether it is a title (`Title`), a table (`Table`), an image
> (`Image`), a series of well-formulated sentences (`NarrativeText`), some kind
> of free text (`UncategorizedText`), a part of a list (`ListItem`), and so on. [Learn more](https://docs.unstructured.io/ui/document-elements#element-type).
> - The `element_id` is a unique identifier that Unstructured generates to
> refer to each document element. [Learn more](https://docs.unstructured.io/ui/document-elements#element-id).
> - `metadata` contains supporting details about each document element, such as
> the page number it occurred on, the file it occurred in, and so on. [Learn more](https://docs.unstructured.io/ui/document-elements#metadata).
>
---

7. Some interesting portions of the output include the following. To search for text in the output, right-click anywhere inside the output pane, click **Command Palette**, click **Find**, and then enter the text you want to search for:

   - The Chinese characters on page 3. Search for the text `In StrokeNet, the corresponding`. Notice that the Chinese characters are not interpreted correctly.
   - The formula on page 5. Search for the text `L= LL + Ln`. Notice that the formula's output diverges quite a bit from the original content.
   - Table 2 on page 6. Search for the text `Model Parameters Performance (BLEU)`. Notice that the `text_as_html` output diverges slightly from the original content.
   - Figure 4 on page 8. Search for the text `50 45 40 35`. Notice that the output is not that informative about the original image's content.

8. Now try changing the partitioning strategy to **VLM** and see how the output changes. To do this, run the following cell, which changes the existing workflow's partitioner node to use **VLM** partitioning instead of **High Res**.

   [Learn more about this code](https://docs.unstructured.io/api-reference/workflow/overview#update-a-workflow).

---
>
> 💡 _When would I specify a particular vision language model here? When would I choose one of these models over another?_
>
> A _vision language model_ (VLM) is designed to use sophisticated AI
> techniques and logic to combine advanced image and text understanding,
> resulting in more accurate and contextually-rich output.
>
> The following code uses a default VLM. As VLMs are constantly being released
> and improved, Unstructured is always
> adding to and updating its list of supported VLMs. If you aren't getting
> consistent results with one VLM for a particular set of files, switching over
> to another one might improve your results, depending on that VLM's
> capabilities and the sample data that is was trained on.

In [ ]:
from unstructured_client import UnstructuredClient
from unstructured_client.models.shared import (
    WorkflowNode,
    UpdateWorkflow,
    WorkflowType
)
from unstructured_client.models.operations import UpdateWorkflowRequest

def update_workflow_partitioner_type(WORKFLOW_ID, type):
    with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
        partition_node = None

        if type == "vlm":
            partition_node = WorkflowNode(
                name="Partitioner",
                subtype="vlm",
                type="partition",
                settings={
                    "is_dynamic": "false",
                    "allow_fast": True
                }
            )
        elif type == "hi_res":
            partition_node = WorkflowNode(
                name="Partitioner",
                subtype="unstructured_api",
                type="partition",
                settings={
                    "strategy": "hi_res"
                }
            )
        else:
            print(f"ERROR: Cannot change workflow: Unrecognized partitioner type '{type}'.")
            return

        workflow = UpdateWorkflow(
            workflow_type=WorkflowType.CUSTOM,
            workflow_nodes=[
                partition_node
            ]
        )

        response = client.workflows.update_workflow(
            request=UpdateWorkflowRequest(
                workflow_id=WORKFLOW_ID,
                update_workflow=workflow
            )
        )

        print(f"Workflow details:\n---\n{response.workflow_information.model_dump_json(indent=4)}")

update_workflow_partitioner_type(WORKFLOW_ID, "vlm")

Workflow details:
---
{
    "created_at": "2025-09-30T18:04:14.558164Z",
    "destinations": [],
    "id": "ca99f800-8746-43e1-8f92-b6e36dbd22f8",
    "name": "api-demo-workflow-2025-09-30-18-04-14",
    "sources": [],
    "status": "active",
    "workflow_nodes": [
        {
            "name": "Partitioner",
            "subtype": "vlm",
            "type": "partition",
            "id": "bb1600ad-1350-4ec2-8908-81ff66c50749",
            "settings": {
                "provider": "anthropic",
                "provider_api_key": null,
                "model": "claude-3-5-sonnet-20241022",
                "output_format": "text/html",
                "prompt": null,
                "format_html": true,
                "unique_element_ids": true,
                "is_dynamic": false,
                "allow_fast": true,
                "custom_host_config": null
            }
        }
    ],
    "reprocess_all": true,
    "schedule": {
        "crontab_entries": []
    },
    "updated_at

9. Run the following cell, which runs the workflow as a job and returns the job's ID.

In [ ]:
JOB_ID = run_workflow(CHINESE_CHARACTERS_INPUT_PATH)

Job details:
---
{
    "created_at": "2025-09-30T18:07:53.122512Z",
    "id": "296f04ae-47be-4f6a-8aee-db5dc02648b1",
    "status": "SCHEDULED",
    "workflow_id": "ca99f800-8746-43e1-8f92-b6e36dbd22f8",
    "workflow_name": "api-demo-workflow-2025-09-30-18-04-14",
    "job_type": "ephemeral"
}


10. Run the following cell, which keeps polling for job status until the job is finished.

In [ ]:
job = poll_job_status(JOB_ID)
print(f"Job details:\n---\n{job.model_dump_json(indent=4)}")

Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job successfully completed.
Job details:
---
{
    "id": "296f04ae-47be-4f6a-8aee-db5dc02648b1",
    "node_stats": [
        {
            "failure": 0,
            "in_progress": 1,
            "ready": 0,
            "success": 0,
            "node_name": "Partitioner",
            "node_subtype": "vlm",
            "node_type": "partition"
        }
    ],
    "processing_status": "SUCCESS",
    "message": null
}


11. After the job finishes successfully, run the following cell, which downloads Unstructured's output to the notebook's local `/content/output` subfolder.

In [ ]:
job_info = get_job_download_details(JOB_ID)

download_processed_file(
    job_id=JOB_ID,
    file_id=job_info.output_node_files[0].file_id,
    node_id=job_info.output_node_files[0].node_id,
    output_path=CHINESE_CHARACTERS_OUTPUT_PATH
)

12. Open the new version of this "Chinese characters" output file (if the version from the previous steps is already open, close it, and then open this new version instead) and notice how the quality of the output changes, now that you are using the **VLM** strategy:

    - The Chinese characters on page 3. Search for the text `In StrokeNet, the corresponding`. Notice that the Chinese characters are intepreted correctly.
    - The formula on page 5. Search for the text `match class`. Notice that the formula's output is closer to the original content.
    - Table 2 on page 6. Search for the text `Model Parameters Performance (BLEU)`. Notice that the text_as_html output is closer to the original content.
    - Figure 4 on page 8. Search for the text `Graph showing BLEU scores comparison`. Notice the informative description about the figure.

13. Now try looking at the "Nash letters" PDF file's output by having Unstructured use the **High Res** partitioning strategy. To do this, run the following cell, which changes the existing workflow's partitioner node back to use **High Res** partitioning instead of **VLM**.

In [ ]:
update_workflow_partitioner_type(WORKFLOW_ID, "hi_res")

Workflow details:
---
{
    "created_at": "2025-09-30T18:04:14.558164Z",
    "destinations": [],
    "id": "ca99f800-8746-43e1-8f92-b6e36dbd22f8",
    "name": "api-demo-workflow-2025-09-30-18-04-14",
    "sources": [],
    "status": "active",
    "workflow_nodes": [
        {
            "name": "Partitioner",
            "subtype": "unstructured_api",
            "type": "partition",
            "id": "ab79abf8-a911-47a5-97c4-40ce00dad93b",
            "settings": {
                "strategy": "hi_res",
                "include_page_breaks": false,
                "pdf_infer_table_structure": false,
                "exclude_elements": null,
                "xml_keep_tags": false,
                "encoding": "utf-8",
                "extract_image_block_types": [
                    "image",
                    "table"
                ]
            }
        }
    ],
    "reprocess_all": true,
    "schedule": {
        "crontab_entries": []
    },
    "updated_at": "2025-09-30T18:15:58

14. Run the following cell, which runs the workflow as a job and returns the job's ID.

In [ ]:
JOB_ID = run_workflow(NASH_LETTERS_INPUT_PATH)

Job details:
---
{
    "created_at": "2025-09-30T18:16:03.822526Z",
    "id": "807838fd-b3ea-494c-8ae8-5a155ac8cab4",
    "status": "SCHEDULED",
    "workflow_id": "ca99f800-8746-43e1-8f92-b6e36dbd22f8",
    "workflow_name": "api-demo-workflow-2025-09-30-18-04-14",
    "job_type": "ephemeral"
}


15. Run the following cell, which keeps polling for job status until the job is finished.

In [ ]:
job = poll_job_status(JOB_ID)
print(f"Job details:\n---\n{job.model_dump_json(indent=4)}")

Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job successfully completed.
Job details:
---
{
    "id": "807838fd-b3ea-494c-8ae8-5a155ac8cab4",
    "node_stats": [
        {
            "failure": 0,
            "in_progress": 0,
            "ready": 0,
            "success": 1,
            "node_name": "Partitioner",
            "node_subtype": "unstructured_api",
            "node_type": "partition"
        }
    ],
    "processing_status": "SUCCESS",
    "message": null
}


16. After the job finishes successfully, run the following cell, which downloads Unstructured's output to the notebook's local `/content/output` subfolder.

In [ ]:
job_info = get_job_download_details(JOB_ID)

download_processed_file(
    job_id=JOB_ID,
    file_id=job_info.output_node_files[0].file_id,
    node_id=job_info.output_node_files[0].node_id,
    output_path=NASH_LETTERS_OUTPUT_PATH
)

17. Some interesting portions of the **High Res** output for the `H03-Cryptosystem-proposed-by-Nash.pdf.json` file against the original handwritten and scanned content include the following. Remember, to search for text in the output, right-click anywhere inside the output pane, click **Command Palette**, click **Find**, and then enter the text you want to search for:

    - The handwriting on page 3. Search for the text `Deo Majr`. Notice that the handwriting is not recognized correctly.
    - The mimeograph on page 11. Search for the text `Technicans at this Agency` (note the typo `Technicans`). Notice that the mimeograph contains `18 January 1955`, but the output contains only `January 1955`.
    - The handwritten diagrams on page 13. Search for the text `"page_number": 13`. Notice that no output is generated for the diagrams.

18. Now try changing the partitioning strategy to **VLM** and see how the quality of the output changes. To do this, run the following cell, which changes the existing workflow's partitioner node to use **VLM** partitioning instead of **High Res**.

In [ ]:
update_workflow_partitioner_type(WORKFLOW_ID, "vlm")

Workflow details:
---
{
    "created_at": "2025-09-30T18:04:14.558164Z",
    "destinations": [],
    "id": "ca99f800-8746-43e1-8f92-b6e36dbd22f8",
    "name": "api-demo-workflow-2025-09-30-18-04-14",
    "sources": [],
    "status": "active",
    "workflow_nodes": [
        {
            "name": "Partitioner",
            "subtype": "vlm",
            "type": "partition",
            "id": "4396a0fb-a85f-4a5c-8b89-c7198aa6a19b",
            "settings": {
                "provider": "anthropic",
                "provider_api_key": null,
                "model": "claude-3-5-sonnet-20241022",
                "output_format": "text/html",
                "prompt": null,
                "format_html": true,
                "unique_element_ids": true,
                "is_dynamic": false,
                "allow_fast": true,
                "custom_host_config": null
            }
        }
    ],
    "reprocess_all": true,
    "schedule": {
        "crontab_entries": []
    },
    "updated_at

19. Run the following cell, which runs the workflow as a job and returns the job's ID.



In [ ]:
JOB_ID = run_workflow(NASH_LETTERS_INPUT_PATH)

Job details:
---
{
    "created_at": "2025-09-30T18:25:21.531927Z",
    "id": "95c0c04a-0ad0-4f59-8e1c-4c3b1330cce2",
    "status": "SCHEDULED",
    "workflow_id": "ca99f800-8746-43e1-8f92-b6e36dbd22f8",
    "workflow_name": "api-demo-workflow-2025-09-30-18-04-14",
    "job_type": "ephemeral"
}


20. Run the following cell, which keeps polling for job status until the job is finished.



In [ ]:
job = poll_job_status(JOB_ID)
print(f"Job details:\n---\n{job.model_dump_json(indent=4)}")

Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job is in progress, polling again in 10 seconds...
Job successfully completed.
Job details:
---
{
    "id": "95c0c04a-0ad0-4f59-8e1c-4c3b1330cce2",
    "node_stats": [
        {
            "failure": 0,
            "in_progress": 0,
            "ready": 0,
            "success": 1,
            "node_name": "Partitioner",
            "node_subtype": "vlm",
            "node_type": "partition"
        }
    ],
    "processing_status": "SUCCESS",
    "message": null
}


21. After the job finishes successfully, run the following cell, which downloads Unstructured's output to the notebook's local `/content/output` subfolder.

In [ ]:
job_info = get_job_download_details(JOB_ID)

download_processed_file(
    job_id=JOB_ID,
    file_id=job_info.output_node_files[0].file_id,
    node_id=job_info.output_node_files[0].node_id,
    output_path=NASH_LETTERS_OUTPUT_PATH
)

22. Open the new version of this "Nash letters" output file (if the version from the previous steps is already open, close it, and then open this new version instead). Notice how the output changes, now that you are using the **VLM** strategy:

    - The handwriting on page 3. Search for the text `Dear Major Grosjean`. Notice how well the handwriting is recognized correctly.
    - The mimeograph on page 11. Search for the text `Technicians at this Agency` (note the corrected typo `Technicians`). Notice that the mimoegraph contains `18 January 1955`, and the output now also contains `18 January 1955`.
    - The handwritten diagrams on page 13. Search for the text `graph LR`. Notice that Mermaid representations of the handwritten diagrams are output.

## Step 3: Experiment with enriching

In this step, you add several [enrichments](https://docs.unstructured.io/ui/enriching/overview) to your workflow, such as generating summary descriptions of detected images and tables, HTML representations of detected tables, and detected entities (such as people and organizations) and the inferred relationships among these entities.

---
>
> 💡 _Can you tell me more about what each of these enrichments actually do?_
>
> The _image description_ enrichment generates a summary description of each
> detected image. This can help you to more quickly and easily understand what
> each image is all about without having to stop to manually visualize and
> interpret the image's content yourself. This also provides additional helpful
> context about the image for your RAG apps, agents, and models.
> [Learn more](https://docs.unstructured.io/ui/enriching/image-descriptions).
>
> The _table description_ enrichment generates a summary description of each
> detected table. This can help you to more quickly and easily understand what
> each table is all about without having to stop to manually read through the
> table's content yourself. This also provides additional helpful context about
> the table for your RAG apps, agents, and models.
> [Learn more](https://docs.unstructured.io/ui/enriching/table-descriptions).
>
> The _table-to-HTML_ enrichment generates an HTML representation of each
> detected table. This can help you to more quickly and accurately recreate the
> table's content elsewhere later as needed. This also provides additional
> context about the table's structure for your RAG apps, agents, and models.
> [Learn more](https://docs.unstructured.io/ui/enriching/table-to-html).
>
> The _named entity recognition (NER)_ enrichment generates a list of detected
> entities (such as people and organizations) and the inferred relationships
> among these entities. This provides additional context about these entities'
> types and their relationships for your graph databases, RAG apps, agents, and
> models.
> [Learn more](https://docs.unstructured.io/ui/enriching/ner).
>
---

1. Run the following cell, which changes the existing workflow to use **High Res** partitioning and also adds workflow nodes for generating the image and table summary descriptions, table HTML, and entities and their relationships.

   [Learn more about this code](https://docs.unstructured.io/api-reference/workflow/workflows#enrichment-node).

In [ ]:
from unstructured_client import UnstructuredClient
from unstructured_client.models.shared import (
    WorkflowNode,
    UpdateWorkflow,
    WorkflowType
)
from unstructured_client.models.operations import UpdateWorkflowRequest

with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    partition_node = WorkflowNode(
        name="Partitioner",
        subtype="unstructured_api",
        type="partition",
        settings={
            "strategy": "hi_res"
        }
    )

    image_description_enrichment_node = WorkflowNode(
        name="OpenAI Image Description",
        subtype="openai_image_description",
        type="prompter",
        settings={}
    )

    table_description_enrichment_node = WorkflowNode(
        name="OpenAI Table Description",
        subtype="openai_table_description",
        type="prompter",
        settings={}
    )

    table_to_html_enrichment_node = WorkflowNode(
        name="OpenAI Table to HTML",
        subtype="openai_table2html",
        type="prompter",
        settings={}
    )

    named_entity_recognition_enrichment_node = WorkflowNode(
        name="OpenAI NER",
        subtype="openai_ner",
        type="prompter",
        settings={}
    )

    workflow = UpdateWorkflow(
        workflow_type=WorkflowType.CUSTOM,
        workflow_nodes=[
            partition_node,
            image_description_enrichment_node,
            table_description_enrichment_node,
            table_to_html_enrichment_node,
            named_entity_recognition_enrichment_node
        ]
    )

    response = client.workflows.update_workflow(
        request=UpdateWorkflowRequest(
            workflow_id=WORKFLOW_ID,
            update_workflow=workflow
        )
    )

    print(f"Workflow details:\n---\n{response.workflow_information.model_dump_json(indent=4)}")

Workflow details:
---
{
    "created_at": "2025-09-30T18:04:14.558164Z",
    "destinations": [],
    "id": "ca99f800-8746-43e1-8f92-b6e36dbd22f8",
    "name": "api-demo-workflow-2025-09-30-18-04-14",
    "sources": [],
    "status": "active",
    "workflow_nodes": [
        {
            "name": "OpenAI NER",
            "subtype": "openai_ner",
            "type": "prompter",
            "id": "e80b0e6b-cac5-4425-ae53-d97957eb7468",
            "settings": {}
        },
        {
            "name": "OpenAI Table to HTML",
            "subtype": "openai_table2html",
            "type": "prompter",
            "id": "62d77501-3077-4b0f-a82e-9da8904a5c0e",
            "settings": {}
        },
        {
            "name": "OpenAI Table Description",
            "subtype": "openai_table_description",
            "type": "prompter",
            "id": "6ef107c9-77dd-4561-90b0-5d9dd5920621",
            "settings": {}
        },
        {
            "name": "Partitioner",
            "sub

2. Run the following cell, which runs the workflow as a job and returns the job's ID.

In [ ]:
JOB_ID = run_workflow(CHINESE_CHARACTERS_INPUT_PATH)

3. Run the following cell, which keeps polling for job status until the job is finished.

In [ ]:
job = poll_job_status(JOB_ID)
print(f"Job details:\n---\n{job.model_dump_json(indent=4)}")

4. After the job finishes successfully, run the following cell, which downloads Unstructured's output to the notebook's local `/content/output` subfolder.

In [ ]:
job_info = get_job_download_details(JOB_ID)

download_processed_file(
    job_id=JOB_ID,
    file_id=job_info.output_node_files[0].file_id,
    node_id=job_info.output_node_files[0].node_id,
    output_path=CHINESE_CHARACTERS_OUTPUT_PATH
)

5. Open the new version of this "Chinese characters" output file (if the version from the previous steps is already open, close it, and then open this new version instead). Some interesting portions of the output include the following:

   - The figures on pages 3, 7, and 8. Search for the seven instances of the text `"type": "Image"`. Notice the summary description for each image.
   - The tables on pages 6, 7, 8, 9, and 12. Search for the seven instances of the text `"type": "Table"`. Notice the summary description for each of these tables. Also notice the `text_as_html` field for each of these tables.
   - The identified entities and inferred relationships among them. Search for the text `Zhijun Wang`. Of the eight instances of this name, notice the author's identification as a `PERSON` three times, the author's `published` relationship twice, and the author's `affiliated_with` relationship twice.

## Step 4: Experiment with chunking

In this step, you apply [chunking](https://docs.unstructured.io/ui/chunking) to your workflow. Chunking is the process where Unstructured rearranges the resulting document elements' `text` content into manageable "chunks" to stay within the limits of an AI model and to improve retrieval precision.

---
>
> 💡 _What kind of chunking strategy should I use, and how big should my chunks be?_
>
> Unfortunately, there is no one-size-fits-all answer to this question.
> However, there are some general considerations and guidelines that can help
> you to determine the best chunking strategy and chunk size for your specific
> use case. Be sure of course to also consult the documentation for your target
> AI model and downstream application toolsets.
>
> Is your content primarily organized by title, by page, by interrelated
> subject matter, or none of these? This can help you determine whether a
> by-title, by-page, by-similarity, or basic (by-character) chunking strategy
> is best. (You'll experiment with each of these strategies here later.)
>
> If your chunks are too small, they might lose necessary context, leading to
> the model providing inaccurate, irrelevant, or hallucinated results. On the
> other hand, if your chunks are too large, the model can struggle with the
> sheer volume of information, leading to information overload, diluted
> meaning, and potentially higher processing costs. You should aim to find a
> balance between chunks that are big enough to contain meaningful information,
> while small enough to enable performant applications and low latency
> responses.
>
> For example, smaller chunks of 128 or 256 tokens might be sufficient for
> capturing more granular semantic information, while larger chunks of 512 or
> 1024 tokens might be better for retaining more context. It's important here
> to note that _tokens_ and _characters_ are not the same thing! In terms of
> characters, for English text, a common approximation is 1 token being equal
> to about 3 or 4 characters or three-quarters of a word. Many AI model
> providers publish their own token-to-character calculators online that you
> can use for estimation purposes.
>
> You should experiement with a variety of chunk sizes, taking into account the
> kinds of content, the length and complexity of user queries and agent tasks,
> the intended end use, and of course the limits of the models you are using.
> Try different chunking strategies and sizes with your models and evaluate the
> results for yourself.
>
---

1. Run the following code, which changes the existing workflow to add a workflow node that chunks the document elements' `text` by using a character-based strategy.

   [Learn more about this code](https://docs.unstructured.io/api-reference/workflow/workflows#chunker-node).

---
>
> 💡 _What do each of the following chunking settings do?_
>
> - Contextual chunking (`contextual_chunking_strategy`) prepends
>   chunk-specific explanatory context to each chunk, which has been shown to
>   yield significant improvements in downstream retrieval accuracy.
>   [Learn more]().
> - Include original elements (`include_orig_elements`) outputs into each
>   chunk's `metadata` field's `orig_elements` value the elements that were
>   used to form that particular chunk.
>   [Learn more]().
> - Max characters (`max_characters`) is the "hard" or maximum number of
>   characters that any one chunk can contain. Unstructured cannot exceed this
>   number when forming chunks.
>   [Learn more]().
> - New after n characters (`new_after_n_characters`) is the "soft" or
>   approximate number of characters that any one chunk can contain.
>   Unstructured can exceed this number if needed when forming chunks (but
>   still cannot exceed the max characters setting).
>   [Learn more]().
> - Overlap (`overlap`), when applied (see `overlap_all`), prepends to the
>   current chunk the specified number of characters from the previous chunk,
>   which can help provide additional context about this chunk relative to the
>   previous chunk.
>   [Learn more]().
> - Overlap all (`overlap_all`), applies the `overlap` setting (if greater than
>   zero) to all chunks. Otherwise, setting `overlap_all` to `False` that the
>   `overlap` setting (if greater than zero) is applied only in edge cases
>   where `normal` chunks cannot be formed by combining whole elements. Set
>   `overlap_all` to `True` with caution as it can introduce noise into
>   otherwise clean semantic units.
>   [Learn more]().
> - Combine text under n characters (`combine_text_under_n_characters`)
>   combines elements from a section into a chunk until a section reaches a
>   length of this many characters.
>   [Learn more]().
> - Multipage sections (`multipage_sections`), when set to `True`, allows
>   sections to span multiple pages.
>   [Learn more]().
> - The similarity threshold (`similarity_threshold`) is a number between `0`
>   and `1` exclusive (`0.01` to `0.99` inclusive). `0.01` means that any two
>   segments of text that are being compared to each other and are considered
>   least identical in semantic meaning to each other are more likely to be
>   combined into the same chunk together, when such combining must occur.
>   `0.99` means that any two segments of text that are being compared to each
>   other and are considered almost identical in semantic meaning to each other
>   are more likely to be combined into the same chunk together, when such
>   combining must occur. Numbers toward `0.01` bias toward least-identical
>   semantic matches, while numbers toward `0.99` bias toward near-identical
>   semantic matches.
>   [Learn more]().

In [ ]:
from unstructured_client import UnstructuredClient
from unstructured_client.models.shared import (
    WorkflowNode,
    UpdateWorkflow,
    WorkflowType
)
from unstructured_client.models.operations import UpdateWorkflowRequest

def update_workflow_chunker_type(
    workflow_id=None,
    subtype=None,
    max_characters=None,
    new_after_n_characters=None,
    combine_text_under_n_chars=None,
    overlap=None,
    overlap_all=False,
    multipage_sections=False,
    include_orig_elements=False,
    contextual_chunking_strategy=None,
    similarity_threshold=None
):
    with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
        partition_node = WorkflowNode(
            name="Partitioner",
            subtype="unstructured_api",
            type="partition",
            settings={
                "strategy": "hi_res"
            }
        )

        image_description_enrichment_node = WorkflowNode(
            name="OpenAI Image Description",
            subtype="openai_image_description",
            type="prompter",
            settings={}
        )

        table_description_enrichment_node = WorkflowNode(
            name="OpenAI Table Description",
            subtype="openai_table_description",
            type="prompter",
            settings={}
        )

        table_to_html_enrichment_node = WorkflowNode(
            name="OpenAI Table to HTML",
            subtype="openai_table2html",
            type="prompter",
            settings={}
        )

        named_entity_recognition_enrichment_node = WorkflowNode(
            name="OpenAI NER",
            subtype="openai_ner",
            type="prompter",
            settings={}
        )

        chunker_node = None

        if subtype == "chunk_by_character":
            chunker_node = WorkflowNode(
                name="Chunker",
                subtype=subtype,
                type="chunk",
                settings={
                    "include_orig_elements": include_orig_elements,
                    "new_after_n_chars": new_after_n_characters,
                    "max_characters": max_characters,
                    "overlap": overlap,
                    "overlap_all": overlap_all,
                    "contextual_chunking_strategy": contextual_chunking_strategy
                }
            )
        elif subtype == "chunk_by_title":
            chunker_node = WorkflowNode(
                name="Chunker",
                subtype=subtype,
                type="chunk",
                settings={
                    "multipage_sections": multipage_sections,
                    "combine_text_under_n_chars": combine_text_under_n_chars,
                    "include_orig_elements": include_orig_elements,
                    "new_after_n_chars": new_after_n_characters,
                    "max_characters": max_characters,
                    "overlap": overlap,
                    "overlap_all": overlap_all,
                    "contextual_chunking_strategy": contextual_chunking_strategy
                }
            )
        elif subtype == "chunk_by_page":
            chunker_node = WorkflowNode(
                name="Chunker",
                subtype="chunk_by_page",
                type="chunk",
                settings={
                    "include_orig_elements": include_orig_elements,
                    "new_after_n_chars": new_after_n_characters,
                    "max_characters": max_characters,
                    "overlap": overlap,
                    "overlap_all": overlap_all,
                    "contextual_chunking_strategy": contextual_chunking_strategy
                }
            )
        elif subtype == "chunk_by_similarity":
            chunker_node = WorkflowNode(
              name="Chunker",
              subtype="chunk_by_similarity",
              type="chunk",
              settings={
                  "include_orig_elements": include_orig_elements,
                  "new_after_n_chars": new_after_n_characters,
                  "max_characters": max_characters,
                  "overlap": overlap,
                  "overlap_all": overlap_all,
                  "contextual_chunking_strategy": contextual_chunking_strategy,
                  "similarity_threshold": similarity_threshold
              }
            )
        else:
            print(f"ERROR: Cannot change workflow: Unrecognized chunker subtype '{subtype}'.")
            return

        workflow = UpdateWorkflow(
            workflow_type=WorkflowType.CUSTOM,
            workflow_nodes=[
                partition_node,
                image_description_enrichment_node,
                table_description_enrichment_node,
                table_to_html_enrichment_node,
                named_entity_recognition_enrichment_node,
                chunker_node
            ]
        )

        response = client.workflows.update_workflow(
            request=UpdateWorkflowRequest(
                workflow_id=WORKFLOW_ID,
                update_workflow=workflow
            )
        )

        print(f"Workflow details:\n---\n{response.workflow_information.model_dump_json(indent=4)}")

update_workflow_chunker_type(
    subtype="chunk_by_character",
    max_characters=500,
    new_after_n_characters=400,
    overlap=50,
    include_orig_elements=True
)

Workflow details:
---
{
    "created_at": "2025-09-30T18:04:14.558164Z",
    "destinations": [],
    "id": "ca99f800-8746-43e1-8f92-b6e36dbd22f8",
    "name": "api-demo-workflow-2025-09-30-18-04-14",
    "sources": [],
    "status": "active",
    "workflow_nodes": [
        {
            "name": "OpenAI NER",
            "subtype": "openai_ner",
            "type": "prompter",
            "id": "e80b0e6b-cac5-4425-ae53-d97957eb7468",
            "settings": {}
        },
        {
            "name": "OpenAI Table to HTML",
            "subtype": "openai_table2html",
            "type": "prompter",
            "id": "62d77501-3077-4b0f-a82e-9da8904a5c0e",
            "settings": {}
        },
        {
            "name": "OpenAI Table Description",
            "subtype": "openai_table_description",
            "type": "prompter",
            "id": "6ef107c9-77dd-4561-90b0-5d9dd5920621",
            "settings": {}
        },
        {
            "name": "Partitioner",
            "sub

2. Run the following cell, which runs the workflow as a job and returns the job's ID.


In [ ]:
JOB_ID = run_workflow(CHINESE_CHARACTERS_INPUT_PATH)

3. Run the following cell, which keeps polling for job status until the job is finished.

In [ ]:
job = poll_job_status(JOB_ID)
print(f"Job details:\n---\n{job.model_dump_json(indent=4)}")

4. After the job finishes successfully, run the following cell, which downloads Unstructured's output to the notebook's local `/content/output` subfolder.

In [ ]:
job_info = get_job_download_details(JOB_ID)

download_processed_file(
    job_id=JOB_ID,
    file_id=job_info.output_node_files[0].file_id,
    node_id=job_info.output_node_files[0].node_id,
    output_path=CHINESE_CHARACTERS_OUTPUT_PATH
)

5. Open the new version of this "Chinese characters" output file (if the version from the previous steps is already open, close it, and then open this new version instead). To explore the chunker's results, search for the text `"type": "CompositeElement"`.
6. Now run the following cell, which changes the existing workflow's chunker node to chunk the document elements' text by using a title-based strategy.

In [ ]:
update_workflow_chunker_type(
    subtype="chunk_by_title",
    max_characters=500,
    new_after_n_characters=400,
    overlap=50,
    include_orig_elements=True
)

Workflow details:
---
{
    "created_at": "2025-09-30T18:04:14.558164Z",
    "destinations": [],
    "id": "ca99f800-8746-43e1-8f92-b6e36dbd22f8",
    "name": "api-demo-workflow-2025-09-30-18-04-14",
    "sources": [],
    "status": "active",
    "workflow_nodes": [
        {
            "name": "OpenAI NER",
            "subtype": "openai_ner",
            "type": "prompter",
            "id": "e80b0e6b-cac5-4425-ae53-d97957eb7468",
            "settings": {}
        },
        {
            "name": "OpenAI Table to HTML",
            "subtype": "openai_table2html",
            "type": "prompter",
            "id": "62d77501-3077-4b0f-a82e-9da8904a5c0e",
            "settings": {}
        },
        {
            "name": "OpenAI Table Description",
            "subtype": "openai_table_description",
            "type": "prompter",
            "id": "6ef107c9-77dd-4561-90b0-5d9dd5920621",
            "settings": {}
        },
        {
            "name": "Partitioner",
            "sub

7. Run the following cell, which runs the workflow as a job and returns the job's ID.

In [ ]:
JOB_ID = run_workflow(CHINESE_CHARACTERS_INPUT_PATH)

8. Run the following cell, which keeps polling for job status until the job is finished.

In [ ]:
job = poll_job_status(JOB_ID)
print(f"Job details:\n---\n{job.model_dump_json(indent=4)}")

9. After the job finishes successfully, run the following cell, which downloads Unstructured's output to the notebook's local `/content/output` subfolder.

In [ ]:
job_info = get_job_download_details(JOB_ID)

download_processed_file(
    job_id=JOB_ID,
    file_id=job_info.output_node_files[0].file_id,
    node_id=job_info.output_node_files[0].node_id,
    output_path=CHINESE_CHARACTERS_OUTPUT_PATH
)

10. Open the new version of this "Chinese characters" output file (if the version from the previous steps is already open, close it, and then open this new version instead). To explore the chunker's results, search for the text `"type": "CompositeElement"`. Notice that the lengths of some of the chunks that immediately precede titles might be shortened due to the presence of the title impacting the chunk's size.
11. Now run the following cell, which changes the existing workflow's chunker node to chunk the document elements' text by using a page-based strategy.

In [ ]:
update_workflow_chunker_type(
    subtype="chunk_by_page",
    max_characters=500,
    new_after_n_characters=400,
    overlap=50,
    include_orig_elements=True
)

Workflow details:
---
{
    "created_at": "2025-09-30T18:04:14.558164Z",
    "destinations": [],
    "id": "ca99f800-8746-43e1-8f92-b6e36dbd22f8",
    "name": "api-demo-workflow-2025-09-30-18-04-14",
    "sources": [],
    "status": "active",
    "workflow_nodes": [
        {
            "name": "OpenAI NER",
            "subtype": "openai_ner",
            "type": "prompter",
            "id": "e80b0e6b-cac5-4425-ae53-d97957eb7468",
            "settings": {}
        },
        {
            "name": "OpenAI Table to HTML",
            "subtype": "openai_table2html",
            "type": "prompter",
            "id": "62d77501-3077-4b0f-a82e-9da8904a5c0e",
            "settings": {}
        },
        {
            "name": "OpenAI Table Description",
            "subtype": "openai_table_description",
            "type": "prompter",
            "id": "6ef107c9-77dd-4561-90b0-5d9dd5920621",
            "settings": {}
        },
        {
            "name": "Partitioner",
            "sub

12. Run the following cell, which runs the workflow as a job and returns the job's ID.

In [ ]:
JOB_ID = run_workflow(CHINESE_CHARACTERS_INPUT_PATH)

13. Run the following cell, which keeps polling for job status until the job is finished.

In [ ]:
job = poll_job_status(JOB_ID)
print(f"Job details:\n---\n{job.model_dump_json(indent=4)}")

14. After the job finishes successfully, run the following cell, which downloads Unstructured's output to the notebook's local `/content/output` subfolder.

In [ ]:
job_info = get_job_download_details(JOB_ID)

download_processed_file(
    job_id=JOB_ID,
    file_id=job_info.output_node_files[0].file_id,
    node_id=job_info.output_node_files[0].node_id,
    output_path=CHINESE_CHARACTERS_OUTPUT_PATH
)

15. Open the new version of this "Chinese characters" output file (if the version from the previous steps is already open, close it, and then open this new version instead). To explore the chunker's results, search for the text `"type": "CompositeElement"`. Notice that the lengths of some of the chunks that immediately precede page breaks might be shortened due to the presence of the page break impacting the chunk's size.
16. Now run the following cell, which changes the existing workflow's chunker node to chunk the document elements' text by using a similarity-based strategy.

In [ ]:
update_workflow_chunker_type(
    subtype="chunk_by_similarity",
    max_characters=500,
    similarity_threshold=0.99,
    include_orig_elements=True
)

Workflow details:
---
{
    "created_at": "2025-09-30T18:04:14.558164Z",
    "destinations": [],
    "id": "ca99f800-8746-43e1-8f92-b6e36dbd22f8",
    "name": "api-demo-workflow-2025-09-30-18-04-14",
    "sources": [],
    "status": "active",
    "workflow_nodes": [
        {
            "name": "OpenAI NER",
            "subtype": "openai_ner",
            "type": "prompter",
            "id": "e80b0e6b-cac5-4425-ae53-d97957eb7468",
            "settings": {}
        },
        {
            "name": "OpenAI Table to HTML",
            "subtype": "openai_table2html",
            "type": "prompter",
            "id": "62d77501-3077-4b0f-a82e-9da8904a5c0e",
            "settings": {}
        },
        {
            "name": "OpenAI Table Description",
            "subtype": "openai_table_description",
            "type": "prompter",
            "id": "6ef107c9-77dd-4561-90b0-5d9dd5920621",
            "settings": {}
        },
        {
            "name": "Partitioner",
            "sub

17. Run the following cell, which runs the workflow as a job and returns the job's ID.

In [ ]:
JOB_ID = run_workflow(CHINESE_CHARACTERS_INPUT_PATH)

18. Run the following cell, which keeps polling for job status until the job is finished.

In [ ]:
job = poll_job_status(JOB_ID)
print(f"Job details:\n---\n{job.model_dump_json(indent=4)}")

19. After the job finishes successfully, run the following cell, which downloads Unstructured's output to the notebook's local `/content/output` subfolder.

In [ ]:
job_info = get_job_download_details(JOB_ID)

download_processed_file(
    job_id=JOB_ID,
    file_id=job_info.output_node_files[0].file_id,
    node_id=job_info.output_node_files[0].node_id,
    output_path=CHINESE_CHARACTERS_OUTPUT_PATH
)

20. Open the new version of this "Chinese characters" output file (if the version from the previous steps is already open, close it, and then open this new version instead). To explore the chunker's results, search for the text `"type": "CompositeElement"`. Notice that the lengths of many of the chunks fall well short of the `max_characters` limit. This is because a similarity threshold of `0.99` means that only sentences or text segments with a near-perfect semantic match will be grouped together into the same chunk. This is an extremely high threshold, resulting in very short, highly specific chunks of text.
21. Now run the following cell, which changes the existing workflow's chunker node to use a different similarity threshold.

In [ ]:
update_workflow_chunker_type(
    subtype="chunk_by_similarity",
    max_characters=500,
    similarity_threshold=0.01,
    include_orig_elements=True
)

22. Run the following cell, which runs the workflow as a job and returns the job's ID.

In [ ]:
JOB_ID = run_workflow(CHINESE_CHARACTERS_INPUT_PATH)

23. Run the following cell, which keeps polling for job status until the job is finished.

In [ ]:
job = poll_job_status(JOB_ID)
print(f"Job details:\n---\n{job.model_dump_json(indent=4)}")

24. After the job finishes successfully, run the following cell, which downloads Unstructured's output to the notebook's local `/content/output` subfolder.

In [ ]:
job_info = get_job_download_details(JOB_ID)

download_processed_file(
    job_id=JOB_ID,
    file_id=job_info.output_node_files[0].file_id,
    node_id=job_info.output_node_files[0].node_id,
    output_path=CHINESE_CHARACTERS_OUTPUT_PATH
)

25. Open the new version of this "Chinese characters" output file (if the version from the previous steps is already open, close it, and then open this new version instead). To explore the chunker's results, search for the text `"type": "CompositeElement"`. Notice now that many of the chunks will now come closer to the `max_characters` limit. This is because a similarity threshold of `0.01` provides an extreme tolerance of differences between pieces of text, grouping almost anything together.

## Step 6: Experiment with embedding

In this step, you generate [embeddings](https://docs.unstructured.io/ui/embedding) for your workflow. Embeddings are vectors of numbers that represent various aspects of the text that is extracted by Unstructured. These vectors are stored or "embedded" next to the text itself in a vector store or vector database. Chatbots, agents, and other AI solutions can use these vector embeddings to more efficiently and effectively find, analyze, and use the associated text. These vector embeddings are generated by an embedding model that is provided by an embedding provider. For the best embedding model to apply to your use case, see the documentation for your target downstream application toolsets.


1. Run the following cell, which changes the existing workflow to add a workflow node that uses the `text-embedding-3-small` embedding model provided by Azure OpenAI.

In [ ]:
from unstructured_client import UnstructuredClient
from unstructured_client.models.shared import (
    WorkflowNode,
    UpdateWorkflow,
    WorkflowType
)
from unstructured_client.models.operations import UpdateWorkflowRequest

with UnstructuredClient(api_key_auth=UNSTRUCTURED_API_KEY) as client:
    partition_node = WorkflowNode(
        name="Partitioner",
        subtype="unstructured_api",
        type="partition",
        settings={
            "strategy": "hi_res"
        }
    )

    image_description_enrichment_node = WorkflowNode(
        name="OpenAI Image Description",
        subtype="openai_image_description",
        type="prompter",
        settings={}
    )

    table_description_enrichment_node = WorkflowNode(
        name="OpenAI Table Description",
        subtype="openai_table_description",
        type="prompter",
        settings={}
    )

    table_to_html_enrichment_node = WorkflowNode(
        name="OpenAI Table to HTML",
        subtype="openai_table2html",
        type="prompter",
        settings={}
    )

    named_entity_recognition_enrichment_node = WorkflowNode(
        name="OpenAI NER",
        subtype="openai_ner",
        type="prompter",
        settings={}
    )

    chunker_node = WorkflowNode(
        name="Chunker",
        subtype="chunk_by_character",
        type="chunk",
        settings={
            "max_characters": 500,
            "new_after_n_chars": 400,
            "overlap": 50,
            "include_orig_elements": True
        }
    )

    embedder_node = WorkflowNode(
        name="Embedder",
        subtype="azure_openai",
        type="embed",
        settings={
            "model_name": "text-embedding-3-small"
        }
    )

    workflow = UpdateWorkflow(
        workflow_type=WorkflowType.CUSTOM,
        workflow_nodes=[
            partition_node,
            image_description_enrichment_node,
            table_description_enrichment_node,
            table_to_html_enrichment_node,
            named_entity_recognition_enrichment_node,
            chunker_node,
            embedder_node
        ]
    )

    response = client.workflows.update_workflow(
        request=UpdateWorkflowRequest(
            workflow_id=WORKFLOW_ID,
            update_workflow=workflow
        )
    )

    print(f"Workflow details:\n---\n{response.workflow_information.model_dump_json(indent=4)}")

Workflow details:
---
{
    "created_at": "2025-09-30T18:04:14.558164Z",
    "destinations": [],
    "id": "ca99f800-8746-43e1-8f92-b6e36dbd22f8",
    "name": "api-demo-workflow-2025-09-30-18-04-14",
    "sources": [],
    "status": "active",
    "workflow_nodes": [
        {
            "name": "OpenAI NER",
            "subtype": "openai_ner",
            "type": "prompter",
            "id": "e80b0e6b-cac5-4425-ae53-d97957eb7468",
            "settings": {}
        },
        {
            "name": "OpenAI Table to HTML",
            "subtype": "openai_table2html",
            "type": "prompter",
            "id": "62d77501-3077-4b0f-a82e-9da8904a5c0e",
            "settings": {}
        },
        {
            "name": "OpenAI Table Description",
            "subtype": "openai_table_description",
            "type": "prompter",
            "id": "6ef107c9-77dd-4561-90b0-5d9dd5920621",
            "settings": {}
        },
        {
            "name": "Partitioner",
            "sub

2. Run the following cell, which runs the workflow as a job and returns the job's ID.

In [ ]:
JOB_ID = run_workflow(CHINESE_CHARACTERS_INPUT_PATH)

3. Run the following cell, which keeps polling for job status until the job is finished.

In [ ]:
job = poll_job_status(JOB_ID)
print(f"Job details:\n---\n{job.model_dump_json(indent=4)}")

4. After the job finishes successfully, run the following cell, which downloads Unstructured's output to the notebook's local `/content/output` subfolder.

In [ ]:
job_info = get_job_download_details(JOB_ID)

download_processed_file(
    job_id=JOB_ID,
    file_id=job_info.output_node_files[0].file_id,
    node_id=job_info.output_node_files[0].node_id,
    output_path=CHINESE_CHARACTERS_OUTPUT_PATH
)

5. Open the new version of this "Chinese characters" output file (if the version from the previous steps is already open, close it, and then open this new version instead). To explore the embeddings, search for the text `"embeddings"`.

---
>
> 💡 _What do all of these numbers mean?_
>
> All by themselves, the numbers in the `embeddings` field of the output have
> no human-interpretable meaning on their own. However, when combined with the
> specific text that these numbers are associated with, and the embedding
> model's logic that was used to generate these numbers, the numbers in the
> `embeddings` field are extremely powerful when leveraged by downstream
> chatbots, agents, and other AI solutions.
>
> These numbers typically represent complex, abstract attributes about the text
> that are known only to the embedding model that generated these numbers.
> These attributes can be about the text's overall sentiment, intent, subject,
> semantic meaning, grammatical function, relationships between words, or any
> number of other things that the model is good at figuring out. This is why
> the embedding model you choose here must be the exact same embedding model
> that you use in any related chatbot, agent, or other AI solution that relies
> on these numbers. Otherwise, the numbers that are generated here will not
> have the same meaning downstream as well. Also, the number of dimensions (or
> the number of numbers in the `embeddings` field) you choose here must also be
> the exact same number of dimensions downstream as well.
>
> To repeat, the name and number of dimensions for the embedding model you
> choose here must be the exact same name and number of dimensions for the
> embedding model you use in your related downstream chatbots, agents, and
> other AI solutions that rely on this particular text and its associated
> embeddings that were generated here.

## Next steps

Congratulations! You now have an Unstructured workflow that partitions, enriches, chunks, and embeds your source documents, producing context-rich data that is ready for retrieval-augmented generation (RAG), agentic AI, and model fine-tuning.

Right now, your workflow only accepts one local file at a time for input. Your workflow also only sends Unstructured's processed data to your screen or to be saved locally as a JSON file. You can modify your workflow to accept multiple files and data from&mdash;and send Unstructured's processed data to&mdash;one or more file storage locations, databases, and vector stores. To learn how to do this, try the [Dropbox-to-Pinecone Connector API Quickstart for Unstructured](https://colab.research.google.com/github/Unstructured-IO/notebooks/blob/main/notebooks/Dropbox_To_Pinecone_Connector_Quickstart.ipynb) notebook.

Unstructured also offers a user interface (UI), which allows you to use a graphical user interface to work with Unstructured instead of only with the API. For details, see the [Unstructured UI Overview](https://docs.unstructured.io/ui/overview).

If you are not able to complete any of the preceding quickstarts, contact Unstructured Support at [support@unstructured.io](mailto:support@unstructured.io).